# 10 - Pretraining CamemBERT

Run the script below with :
- train_file: the local absolute path to paper-ner-bench-das22/dataset/unsupervised_pretraining/10-normalized/all.txt
- output_dir: where to save the pretrained model

In [1]:
""" Loads the configuration """

# Set to 1/true/ to set the logging level of nerlogger to DEBUG 
# and save the the spacy datasets as TXT along with the .spacy file
#  for easier debug of the training set generation.
%env DEBUG=1

# If True, activates a set of assertions in the notebooks to ensure
# that the scripts runs with the parameters used in the paper.
%env AS_IN_THE_PAPER = True

import util.config as config

config.show()

env: DEBUG=1
env: AS_IN_THE_PAPER=True


18/05/2022 03:32:38 ; INFO ; ======= CONFIGURATION =======
18/05/2022 03:32:38 ; INFO ; BASEDIR: /home/bertrand/paper-ner-bench-das22
18/05/2022 03:32:38 ; INFO ; Input datasets will be loaded from DATASETDIR /home/bertrand/paper-ner-bench-das22/dataset
18/05/2022 03:32:38 ; INFO ; Training data and models will be saved to NERDIR /home/bertrand/paper-ner-bench-das22/src/ner
18/05/2022 03:32:38 ; INFO ; Debug mode is ON
18/05/2022 03:32:38 ; INFO ; Random seed: 42
18/05/2022 03:32:38 ; INFO ; Enable reproducibility checks: True
18/05/2022 03:32:38 ; INFO ; ============================


In [2]:
""" Import all modules at once """

# General imports
import nltk
import gzip

# NER imports
from util.as_in_the_paper import assert_expected


## 11 - Creation of the pre-training set from raw entries extracted with OCR

In [8]:
import nltk

pretraining_set_path = config.DATASETDIR / "unsupervised_pretraining/10-normalized"

# Filter the normalized pre-training 
# Keep only entries containing at least MIN_WORDS_PER_ENTRY words
MIN_WORDS_PER_ENTRY = 7 # Keep simples entries like "Morel abbé ,Tournon, 14."

assert_expected(actual=MIN_WORDS_PER_ENTRY, expected=7)

nltk.download("punkt")
number_of_entries = 0
valid_entries = []

with gzip.open(pretraining_set_path / "all.txt.gz", "rt") as all_txt:
    for entry in all_txt:
        entry = entry.strip() # Sanitize
        words = nltk.word_tokenize(entry, language="fr", preserve_line=True)
        if len(words) >= MIN_WORDS_PER_ENTRY:
            valid_entries.append(entry)
        number_of_entries += 1
        
with open(pretraining_set_path / "all_filtered.txt", "w") as wfp:
    wfp.write("\n".join(valid_entries))

print("Valid entries:%d, %f.2 percent of the total" % (len(valid_entries),len(valid_entries) / number_of_entries))

[nltk_data] Downloading package punkt to /home/bertrand/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Valid entries:845014, 0.808105.2 percent of the total


## 12 - Pretraining CamemBERT on domain texts

In [ ]:
!python pretrain_camembert.py --model_name_or_path "Jean-Baptiste/camembert-ner" --train_file "${PRETRAINING_SET_PATH}/unsupervised_pretraining/10-normalized/all_filtered.txt" --do_train --do_eval --line_by_line --output_dir "${BASE}/10-camembert_pretrained_model" --save_total_limit 1 --load_best_model_at_end True --evaluation_strategy "steps" --save_strategy "steps"